In [1]:
path_data = '/data'

In [2]:
import pandas as pd
import os
import numpy as np
import torch
from pkg_resources import packaging
from PIL import Image
import tqdm
from pathlib import Path
print("Torch version:", torch.__version__)

Torch version: 1.10.2+cu113


In [3]:
data_unfiltered = pd.read_csv("/".join([path_data, "images_v2.csv"]))
display(data_unfiltered)

,id,country.name,country.id,region.id,type,imageRelPath,topics,place,income
0,5d4be6cecf0b3a0f3f344586,Burundi,bi,af,image,assets/5d4be6cecf0b3a0f3f344586/5d4be6cecf0b3a...,icons,butoyi,26.994581
1,5d4bf31ccf0b3a0f3f359814,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f359814/5d4bf31ccf0b3a...,Family snapshots,butoyi,26.994581
2,5d4bf31ccf0b3a0f3f35982a,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f35982a/5d4bf31ccf0b3a...,Cutlery,butoyi,26.994581
3,5d4bf31ccf0b3a0f3f35982e,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f35982e/5d4bf31ccf0b3a...,Family,butoyi,26.994581
4,5d4bf31ccf0b3a0f3f359830,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f359830/5d4bf31ccf0b3a...,Place where eating dinner,butoyi,26.994581
...,...,...,...,...,...,...,...,...,...
38474,5ec4f5513f62767d97a47324,France,fr,eu,image,assets/5ec4f5513f62767d97a47324/5ec4f5513f6276...,Bed,larriere,19671.000000
38475,5ec4f5513f62767d97a47325,France,fr,eu,image,assets/5ec4f5513f62767d97a47325/5ec4f5513f6276...,Bathroom/Toilet,larriere,19671.000000
38476,5ec4f5523f62767d97a47327,France,fr,eu,image,assets/5ec4f5523f62767d97a47327/5ec4f5523f6276...,Armchair,larriere,19671.000000
38477,5ec4f5523f62767d97a47328,France,fr,eu,image,assets/5ec4f5523f62767d97a47328/5ec4f5523f6276...,Armchair,larriere,19671.000000


### Removing Subjective topics

In [4]:
subjective_topics = ['Favorite home decorations', 'Favourite item in kitchen', 'Favourite sports clubs', 'How the most loved item is used', 'icons', 'Idols', 'Latest furniture bought', ' looking over the shoulder', 'Most loved item', 'Most loved toy', 'Most played songs on the radio', 'Music idol', 'Next big thing you are planning to buy', 'Playing with most loved toy', 'Thing I dream about having', 'Things I wish I had', 'Using most loved item', 'Youth culture', 'What I wish I could buy']

In [5]:
unique_topics = data_unfiltered['topics'].unique()


In [6]:
Subjective_topic_list = []

for topic in unique_topics:
    topic_ = topic.replace('/ ', ', ')
    topic_split = topic_.split(',')
    for split in topic_split:
        if split in subjective_topics:
            Subjective_topic_list.append(topic)
                

In [7]:
data = data_unfiltered[~data_unfiltered['topics'].isin(Subjective_topic_list)]
display(data)

,id,country.name,country.id,region.id,type,imageRelPath,topics,place,income
1,5d4bf31ccf0b3a0f3f359814,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f359814/5d4bf31ccf0b3a...,Family snapshots,butoyi,26.994581
2,5d4bf31ccf0b3a0f3f35982a,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f35982a/5d4bf31ccf0b3a...,Cutlery,butoyi,26.994581
3,5d4bf31ccf0b3a0f3f35982e,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f35982e/5d4bf31ccf0b3a...,Family,butoyi,26.994581
4,5d4bf31ccf0b3a0f3f359830,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f359830/5d4bf31ccf0b3a...,Place where eating dinner,butoyi,26.994581
5,5d4bf31dcf0b3a0f3f35983c,Burundi,bi,af,image,assets/5d4bf31dcf0b3a0f3f35983c/5d4bf31dcf0b3a...,Plate of food,butoyi,26.994581
...,...,...,...,...,...,...,...,...,...
38474,5ec4f5513f62767d97a47324,France,fr,eu,image,assets/5ec4f5513f62767d97a47324/5ec4f5513f6276...,Bed,larriere,19671.000000
38475,5ec4f5513f62767d97a47325,France,fr,eu,image,assets/5ec4f5513f62767d97a47325/5ec4f5513f6276...,Bathroom/Toilet,larriere,19671.000000
38476,5ec4f5523f62767d97a47327,France,fr,eu,image,assets/5ec4f5523f62767d97a47327/5ec4f5523f6276...,Armchair,larriere,19671.000000
38477,5ec4f5523f62767d97a47328,France,fr,eu,image,assets/5ec4f5523f62767d97a47328/5ec4f5523f6276...,Armchair,larriere,19671.000000


In [8]:
data.to_csv('data/images_v2_no_sub_topics.csv')

## Get CLIP embeddings

In [9]:
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-0agb5p64
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-0agb5p64
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33


In [10]:
import clip
clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [11]:
model, preprocess = clip.load("ViT-B/32")

model.cuda().eval()

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          

In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
path_images = [path_data + s for s in list(data['imageRelPath'])]
image_ids1 = [s.split("/")[7] for s in path_images]
image_ids = data['id']
assert(set(image_ids1) == set(image_ids))
# path_images_out = [path_data + img_id + "npy" for img_id in image_ids]
path_images[:2]

## Preprocess Images

In [20]:

outfile = "/".join([path_data, "processed_imgs/"])
if not os.path.exists(outfile):
         os.makedirs(outfile)

In [ ]:
import tqdm
from pathlib import Path

for image_name in tqdm.tqdm(path_images):
    img_id = image_name.split("/")[7]
    outfile = "processed_imgs/" + img_id + ".npy"
    if Path(outfile).is_file():
        print(f"Image {img_id} exists .. moving on")
        continue
    try:
        image = Image.open(image_name)
        image_prep = preprocess(image).to("cpu")
    #     print(img_id, outfile)
        np.save(outfile, image_prep)
    except:
        print(f"Image {img_id} not ok")

  0%|          | 62/36754 [00:25<1:30:38,  6.75it/s]

Image 5fd029321f89a90d2a769c1d not ok
Image 5fd029361f89a90d2a769c47 not ok


  0%|          | 147/36754 [00:53<1:33:08,  6.55it/s]

Image 5d4bf45acf0b3a0f3f35b7f8 not ok
Image 5d4bf45acf0b3a0f3f35b7fa not ok
Image 5d4bf45bcf0b3a0f3f35b812 not ok


  0%|          | 149/36754 [00:53<1:23:20,  7.32it/s]

...

 29%|██▉       | 10690/36754 [49:36<17:41, 24.56it/s]

Image 5fd045def2d9500e4359361f not ok


 29%|██▉       | 10702/36754 [49:37<15:16, 28.41it/s]

Image 5fd045edf2d9500e4359365f not ok
Image 5fd045eef2d9500e43593661 not ok
Image 5fd045f6f2d9500e43593693 not ok
Image 5fd045f7f2d9500e4359369b not ok


 29%|██▉       | 10798/36754 [50:06<2:34:52,  2.79it/s]

In [ ]:
torch.cuda.empty_cache()

## Compute CLIP similarity scores

In [21]:
path_data = 'data/'
path_processed_images = "/".join([path_data, "processed_imgs/"])
path_original_images = '..'

In [22]:
import gc
def flatten_extend(matrix):
    flat_list = []
    for row in matrix:
        flat_list.extend(row)
    return flat_list

In [23]:
for k in tqdm.tqdm(range(13)):
    images = []
    images_prep = []
    image_names_list=[]
    count = 3000
    
    original_images = [path_original_images + s for s in data['imageRelPath']][count*k:count*(k+1)]
    processed_images = [path_processed_images + "/" + img_id + ".npy" for img_id in data['id']][count*k:count*(k+1)]
    image_path_list = []

    imgs_corrupted = []
    for image_path in tqdm.tqdm(processed_images):
        if Path(image_path).is_file():
            image_path_list.append(image_path)
            images_prep.append(torch.from_numpy(np.load(image_path)).to(device))
        else:
            imgs_corrupted.append(image_path)
            #print(f"{image_path} corrupted")

    torch.cuda.empty_cache()
    images_prep = torch.stack(images_prep)
    #print(imgs_corrupted)
    torch.cuda.empty_cache()
    
    
    image_names_list = [image_path.split("/")[-1].split(".")[0] for image_path in image_path_list]
    
    text =  []
    for id_ in image_names_list:
        topic = data[data['id'] == id_]['topics']
        text.append(list(topic))
    text = flatten_extend(text)
    
    temp = "This is a photo of "
    texts = [temp + topic for topic in text]
    
    gc.collect()
    torch.cuda.empty_cache()

    
    with torch.no_grad():
        image_features = model.encode_image(images_prep)
        image_features /= image_features.norm(dim=-1, keepdim=True)

        text_tokens = clip.tokenize([desc for desc in texts]).cuda()
        text_features = model.encode_text(text_tokens).float()
        text_features /= text_features.norm(dim=-1, keepdim=True)
    
    similarity = text_features.cpu().numpy() @ image_features.cpu().numpy().T # dot product of image and text features
    similarity_diag = similarity.diagonal()
    
    results_df = data[data['id'].isin(image_names_list)]
    results_df = results_df.drop(columns=['region.id', 'type', 'imageRelPath', 'place'])
    results_df["CLIP score"] = similarity_diag

    results_df.to_csv(f"data/results_CLIP_{k+1}.csv") 
    print(f"results_CLIP_{k+1} completed")
    
    gc.collect()
    torch.cuda.empty_cache()

 33%|███▎      | 989/3000 [00:46<01:34, 21.32it/s]


 66%|██████▌   | 1976/3000 [01:31<00:52, 19.37it/s]


 98%|█████████▊| 2943/3000 [02:15<00:02, 25.14it/s]


  8%|▊         | 1/13 [02:25<29:09, 145.80s/it]

results_CLIP_1 completed


...

 95%|█████████▍| 2844/3000 [02:12<00:06, 22.87it/s]


 92%|█████████▏| 12/13 [27:33<02:18, 138.50s/it]

results_CLIP_12 completed



100%|██████████| 13/13 [28:11<00:00, 130.09s/it]

results_CLIP_13 completed
